# UKRI funding trends

In [1]:
from discovery_utils.utils import (
    analysis_gtr,
    analysis,
    charts
)

In [2]:
import pandas as pd
from src import PROJECT_DIR

In [3]:
from discovery_utils.getters import gtr
GTR = gtr.GtrGetter()

2024-12-20 12:45:46,283 - discovery_utils.getters.gtr - INFO - Checking for latest version of data in S3 bucket: discovery-iss
2024-12-20 12:45:46,767 - discovery_utils.getters.gtr - INFO - Latest version found: GtR_20241215


In [11]:
included = ['y', 'Maybe?', 'yes']

reviewed_data_df = (
    pd.read_csv(PROJECT_DIR / "data/2024_12_MS/Cybersecurity - Mission studio 2012-12-16 - ukri.csv")
    .rename(columns={"RELEVANT?": "relevant"})
    .query("relevant in @included")
)
matching_ids = reviewed_data_df.id.to_list()

In [13]:
len(matching_ids)

50

In [21]:
matchings_projects_df = (
    GTR.projects_enriched.query("id in @matching_ids")
    .assign(text = lambda x: x.title + " " + x.abstractText)
)
matchings_projects_df[['title', 'url', 'amount', 'start', 'end']]

,title,url,amount,start,end
2401,Hark: A VR based interactive narrative game / ...,https://gtr.ukri.org/projects?ref=10055721,67488,2023-04-01,2023-12-31
4078,Project Safety Net: The risks and opportunitie...,https://gtr.ukri.org/projects?ref=2881747,0,2023-10-01,2027-09-30
8477,Digital Defence &amp; Activism Lessons: Equipp...,https://gtr.ukri.org/projects?ref=AH/T008938/1,79363,2020-08-17,2022-11-11
14526,UnBias: Emancipating Users Against Algorithmic...,https://gtr.ukri.org/projects?ref=EP/N02785X/1,1140593,2016-09-01,2019-05-31
15485,Kidslox 2014; setting boundaries in the age of...,https://gtr.ukri.org/projects?ref=720567,172452,2015-01-01,2015-12-31
17279,2PS - Prevent &amp; Protect Through Support,https://gtr.ukri.org/projects?ref=10047867,382989,2022-10-01,2025-09-30
19029,ISIS: Protecting children in online social net...,https://gtr.ukri.org/projects?ref=EP/F035438/1,430467,2008-06-01,2011-10-18
26148,Using Natural Language Processing and Machine ...,https://gtr.ukri.org/projects?ref=2253445,0,2019-10-01,2022-05-31
26425,"Dynamic, Real time, On-demand Personalisation ...",https://gtr.ukri.org/projects?ref=EP/R033838/1,682696,2018-09-01,2021-02-28
29301,Supporting the use of digital media in researc...,https://gtr.ukri.org/projects?ref=ES/J009814/1,47602,2011-11-01,2012-10-31


In [34]:
matchings_projects_df[['id', 'title', 'url', 'amount', 'start', 'end']].to_csv(PROJECT_DIR / "data/2024_12_MS/gtr_relevant_projects.csv", index=False)

In [32]:
len(matchings_projects_df.query("start > '2013'"))

45

In [28]:
ts_df = (
    analysis_gtr.get_timeseries(matchings_projects_df, 'year', 2014, 2024)
    .assign(amount = lambda df: df.amount / 1_000_000)
)
ts_df

,time_period,year,n_projects,amount,amount_median
0,2014-01-01,2014,0,0.000000,0.0
1,2015-01-01,2015,1,0.172452,172452.0
2,2016-01-01,2016,3,1.176587,35994.0
3,2017-01-01,2017,1,0.532397,532397.0
4,2018-01-01,2018,6,0.698296,0.0
5,2019-01-01,2019,4,0.045452,7475.0
6,2020-01-01,2020,8,0.470046,42024.5
7,2021-01-01,2021,4,5.156993,524455.5
8,2022-01-01,2022,6,1.910968,310377.0
9,2023-01-01,2023,6,0.652636,49744.0


In [29]:
fig = charts.ts_bar(
    ts_df,
    variable='n_projects',
    variable_title="Number of projects",
    category_column="_category",
)
charts.configure_plots(fig, chart_title="")

alt.Chart(...)

In [30]:
fig = charts.ts_bar(
    ts_df,
    variable='amount',
    variable_title="Amount, £ millions",
    category_column="_category",
)
charts.configure_plots(fig, chart_title="")

alt.Chart(...)